In [27]:
import os
import tweepy
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [28]:
consumer_key = os.environ["TWITTER_API_KEY"]
consumer_secret = os.environ["TWITTER_API_KEY_SECRET"]
access_token = os.environ["TWITTER_ACCESS_TOKEN"]
access_token_secret = os.environ["TWITTER_ACCESS_TOKEN_SECRET"]

In [29]:
auth = tweepy.OAuth1UserHandler(
    consumer_key, 
    consumer_secret,
    access_token,
    access_token_secret
    )

api = tweepy.API(auth)
api


In [30]:
me = api.verify_credentials()


In [31]:
search_country = 'India'

In [32]:
def scrape_tweets(search_words_list, date_since, date_until, num_tweets, num_runs, search_country):
    
    tweets_df = pd.DataFrame(columns = ['user_id', 'tweet_id', 'username', 'location', 
                                        'following', 'followers', 'twt_created_at', 'total_tweets', 
                                        'retweet_count', 'text', 'hashtags', 'mentions'])
    
    program_start = time.time()
    
    places = api.search_geo(query=search_country, granularity="country")
    place_id = places[0].id
    count_num = 0
    count_tweet = 0
    for i in range(0, num_runs):
        count_num+=1
        
        start_run = time.time()
        
        for search_words in search_words_list:
            tweets = tweepy.Cursor(api.search_tweets, q=search_words + ' AND ' + ("place:%s" % place_id), lang='en', tweet_mode='extended').items(num_tweets)
            print("count_num = ", count_num)
            
        
            
            tweet_list = [tweet for tweet in tweets]
            print(len(tweet_list))
            no_tweet = 0
            # print(tweet_list)
            for tweet in tweet_list:
                # print(tweet.full_text)
                count_tweet+=1
                # print(tweet.created_at)
                if tweet.created_at.date() < datetime.strptime(date_until, '%Y-%m-%d').date():
                    # print("In here")
                    # print("In here")
                    user_id = tweet.user.id
                    tweet_id = tweet.id
                    username = tweet.user.screen_name
                    # acctdesc = tweet.user.description
                    location = tweet.user.location
                    following = tweet.user.friends_count
                    followers = tweet.user.followers_count
                    twt_created_at = tweet.created_at
                    total_tweets = tweet.user.statuses_count
                    retweet_count = tweet.retweet_count
                    hashtags = tweet.entities['hashtags']
                    mentions = tweet.entities['user_mentions']
                
                    try:    
                        text = tweet.retweeted_status.full_text
                    except AttributeError:
                        text = tweet.full_text
                    # print(text)
                    ith_tweet = [user_id, tweet_id, username, location, 
                                following, followers, twt_created_at ,total_tweets, 
                                retweet_count, text, hashtags, mentions]
                    
                    tweets_df.loc[len(tweets_df)] = ith_tweet
            
                    no_tweet += 1
            print("count_tweet = ", count_tweet)
            time.sleep(920)

        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('Number of tweets scraped for run {} is {}'.format(i+1, no_tweet))
        print('Time take for {} run to complete is {} mins'.format(i+1, duration_run))
    
    
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_tweets.csv'
    tweets_df.to_csv(filename, index=False)
    
    program_end = time.time()
    print('Scraping is completed.')    
    print('Total time taken to scrape is {} minutes.'.format(round(program_end-program_start)/60, 2))
    
    return tweets_df
    


In [33]:
# search_queries = ["hygienic OR medical OR lack OR facilities OR beds OR medicines OR hygienic OR hygiene OR #emergency OR #complaints OR #medicalnegligence OR #patientdeath OR @MoHFW_INDIA OR @PMOIndia OR @ArvindKejriwal OR @mansukhmandviya OR @ANI OR @PTI_News OR @narendramodi"] 

In [34]:
search_words_list = ["#malpractice OR #medicalcare OR #patientdeath OR #unionhealthministry OR #hospital OR @MoHFW_INDIA OR @PMOIndia OR @ArvindKejriwal OR @mansukhmandviya OR @ANI OR @PTI_News OR @narendramodi",
                    "#mcdcomplaint OR #noaction OR #mcd311 OR @MCD_Delhi OR @LtGovDelhi OR @ArvindKejriwal OR @DCkpzon OR @HMOIndia OR @AshwaniKumar_92 OR @narendramodi OR @PMOIndia OR @DelhiComplaint OR @SBM_MCD OR medical+protest OR #MedTwitter"]
date_since = '2020-01-01'
date_until = '2022-11-06'
num_tweets = 500
num_runs = 5
time.sleep(920)
scrape_tweets(search_words_list, date_since, date_until, num_tweets, num_runs, search_country)

In [ ]:
# search_words = "emergency OR complaints OR @narendramodi OR condition OR conditions"

# date_since = '2020-01-01'
# date_until = '2022-01-01'
# num_tweets = 2500
# num_runs = 1
# places = api.search_geo(query=search_country, granularity="country")
# place_id = places[0].id

# scrape_tweets(search_words, date_since, date_until, num_tweets, num_runs, search_country, place_id)
